In [ ]:
import os

dataset_url = "https://data.vision.ee.ethz.ch/cvl/rrothe/imdb-wiki/imdb-wiki.tar"

!wget $dataset_url -O imdb-wiki.tar

!tar -xvf imdb-wiki.tar


--2025-01-31 11:38:30--  https://data.vision.ee.ethz.ch/cvl/rrothe/imdb-wiki/imdb-wiki.tar
Resolving data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)... 129.132.52.178, 2001:67c:10ec:36c2::178
Connecting to data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)|129.132.52.178|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2025-01-31 11:38:31 ERROR 404: Not Found.

tar: This does not look like a tar archive
tar: Exiting with failure status due to previous errors


In [ ]:
!ls -lh
!ls -lh imdb_crop

total 4.0K
-rw-r--r-- 1 root root    0 Jan 31 11:38 imdb-wiki.tar
drwxr-xr-x 1 root root 4.0K Jan 29 14:19 sample_data
ls: cannot access 'imdb_crop': No such file or directory


In [ ]:

!wget https://data.vision.ee.ethz.ch/cvl/rrothe/imdb-wiki/static/imdb_crop.tar -O imdb_crop.tar

!tar -xvf imdb_crop.tar


Streaming output truncated to the last 5000 lines.
imdb_crop/98/nm0232998_rm2180228352_1968-5-11_2007.jpg
imdb_crop/98/nm0232998_rm2200475648_1968-5-11_2007.jpg
imdb_crop/98/nm0232998_rm2232200192_1968-5-11_2007.jpg
imdb_crop/98/nm0232998_rm2267584512_1968-5-11_2007.jpg
imdb_crop/98/nm0232998_rm2331093504_1968-5-11_2007.jpg
imdb_crop/98/nm0232998_rm2384117504_1968-5-11_2007.jpg
imdb_crop/98/nm0232998_rm2465311232_1968-5-11_2007.jpg
imdb_crop/98/nm0232998_rm2534190080_1968-5-11_2007.jpg
imdb_crop/98/nm0232998_rm2549197312_1968-5-11_2007.jpg
imdb_crop/98/nm0232998_rm2563156736_1968-5-11_2007.jpg
imdb_crop/98/nm0232998_rm2579933952_1968-5-11_2007.jpg
imdb_crop/98/nm0232998_rm2683415040_1968-5-11_2007.jpg
imdb_crop/98/nm0232998_rm2688802304_1968-5-11_2007.jpg
imdb_crop/98/nm0232998_rm2739898368_1968-5-11_2007.jpg
imdb_crop/98/nm0232998_rm2756675584_1968-5-11_2007.jpg
imdb_crop/98/nm0232998_rm276465664_1968-5-11_2007.jpg
imdb_crop/98/nm0232998_rm2837228288_1968-5-11_2007.jpg
imdb_crop/98/nm

In [ ]:
import scipy.io
import os


dataset_path = "imdb_crop/imdb.mat"
if not os.path.exists(dataset_path):
    raise FileNotFoundError(f"Dataset file not found at {dataset_path}")


data = scipy.io.loadmat(dataset_path)
print("✅ Dataset loaded successfully!")


✅ Dataset loaded successfully!


In [ ]:

import os
import scipy.io
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import img_to_array
import random
from tensorflow.keras.utils import to_categorical


dataset_url = "https://data.vision.ee.ethz.ch/cvl/rrothe/imdb-wiki/static/imdb_crop.tar"
dataset_path = "imdb_crop.tar"

if not os.path.exists("imdb_crop"):
    print("📥 Downloading dataset... (7GB file)")
    !wget $dataset_url -O $dataset_path
    print("📦 Extracting dataset...")
    !tar -xvf $dataset_path
    print("✅ Dataset ready!")

metadata_path = "imdb_crop/imdb.mat"
if not os.path.exists(metadata_path):
    raise FileNotFoundError(f"❌ Dataset file not found: {metadata_path}")

print("📂 Loading dataset metadata...")
data = scipy.io.loadmat(metadata_path)


image_paths = data["imdb"]["full_path"][0][0].squeeze()
dob = data["imdb"]["dob"][0][0].squeeze()
photo_taken = data["imdb"]["photo_taken"][0][0].squeeze()
genders = data["imdb"]["gender"][0][0].squeeze()


ages = photo_taken - (dob // 365.25).astype(int)


valid_idx = (ages > 0) & (ages < 100) & (~np.isnan(genders))
image_paths = image_paths[valid_idx]
ages = ages[valid_idx]
genders = genders[valid_idx].astype(int)


image_paths = [os.path.join("imdb_crop", p[0]) for p in image_paths]

print(f"✅ Loaded {len(image_paths)} images with age & gender labels.")


def load_image(image_path):
    """ Load image, resize to (64,64), normalize, and return array. """
    try:
        img = cv2.imread(image_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (64, 64))
        img = img_to_array(img) / 255.0
        return img
    except:
        return None


num_samples = 10_000
selected_indices = random.sample(range(len(image_paths)), num_samples)
selected_paths = [image_paths[i] for i in selected_indices]
selected_ages = [ages[i] for i in selected_indices]
selected_genders = [genders[i] for i in selected_indices]

X = []
y_age = []
y_gender = []
for i, path in enumerate(selected_paths):
    img = load_image(path)
    if img is not None:
        X.append(img)
        y_age.append(selected_ages[i])
        y_gender.append(selected_genders[i])

X = np.array(X)
y_age = np.array(y_age).astype(np.float32)
y_gender = to_categorical(y_gender, 2)
print(f"✅ Loaded {len(X)} images for training.")

input_shape = (64, 64, 3)

inputs = tf.keras.Input(shape=input_shape)

x = Conv2D(32, (3,3), activation="relu")(inputs)
x = MaxPooling2D(pool_size=(2,2))(x)
x = Conv2D(64, (3,3), activation="relu")(x)
x = MaxPooling2D(pool_size=(2,2))(x)
x = Flatten()(x)
x = Dense(128, activation="relu")(x)
x = Dropout(0.5)(x)

age_output = Dense(1, activation="linear", name="age_output")(x)

gender_output = Dense(2, activation="softmax", name="gender_output")(x)

model = tf.keras.Model(inputs=inputs, outputs=[age_output, gender_output])


model.compile(
    optimizer="adam",
    loss={"age_output": "mse", "gender_output": "categorical_crossentropy"},
    metrics={"age_output": "mae", "gender_output": "accuracy"},
)

print("🚀 Training the model...")
model.fit(X, {"age_output": y_age, "gender_output": y_gender}, epochs=10, batch_size=32, validation_split=0.2)

model.save("age_gender_model.h5")
print("✅ Model training complete & saved!")

def predict_age_gender(image_path):
    """Predicts age and gender from an image."""
    img = load_image(image_path)
    if img is not None:
        img = np.expand_dims(img, axis=0)
        predicted_age, predicted_gender = model.predict(img)

        gender_label = "Male" if np.argmax(predicted_gender) == 1 else "Female"

        print(f"🎯 Predicted Age: {predicted_age[0][0]:.1f} years")
        print(f"🎯 Predicted Gender: {gender_label}")
    else:
        print("❌ Error loading image!")

test_img = random.choice(selected_paths)
print(f"🖼 Testing image: {test_img}")
predict_age_gender(test_img)


📂 Loading dataset metadata...
✅ Loaded 451567 images with age & gender labels.
✅ Loaded 10000 images for training.
🚀 Training the model...
Epoch 1/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 8s 15ms/step - age_output_loss: 325.7394 - age_output_mae: 13.8320 - gender_output_accuracy: 0.5289 - gender_output_loss: 2.8251 - loss: 328.5644 - val_age_output_loss: 186.6768 - val_age_output_mae: 10.8918 - val_gender_output_accuracy: 0.5925 - val_gender_output_loss: 0.8298 - val_loss: 186.5212
Epoch 2/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - age_output_loss: 201.7864 - age_output_mae: 11.0439 - gender_output_accuracy: 0.5075 - gender_output_loss: 1.1057 - loss: 202.8920 - val_age_output_loss: 175.2474 - val_age_output_mae: 10.0529 - val_gender_output_accuracy: 0.4075 - val_gender_output_loss: 0.7536 - val_loss: 174.6705
Epoch 3/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - age_output_loss: 191.2293 - age_output_mae: 10.8295 - gender_output_accuracy: 0.5470 - gender_output_loss: 0.7090 - loss: 191.9384

✅ Model training complete & saved!
🖼 Testing image: imdb_crop/38/nm0000638_rm3048517632_1931-3-22_2006.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 473ms/step
🎯 Predicted Age: 43.6 years
🎯 Predicted Gender: Male
